In [ ]:
import torch
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from unsloth import FastLanguageModel
# import datasets
# import numpy as np
# from datasets import load_dataset
import pandas as pd

# import transformers
from transformers import (
    # AutoModelForSeq2SeqLM,
    AutoModelForCausalLM,
    AutoTokenizer,
)

# from transformers import Pipeline
# from transformers.pipelines.pt_utils import KeyDataset
from transformers import BitsAndBytesConfig
from peft import PeftModel, PeftConfig

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-06-01 09:08:03.004069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-01 09:08:09.552773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


🦥 Unsloth Zoo will now patch everything to make training faster!


In [1]:
# I use unsloth with qwen3

# Your adapter repo or local dir
# peft_model_id = "mika5883/ru_qwen_gec" # or your output_dir path
# peft_model_id = 'mika5883/ru_qwen_gec_Ag_art'
# # peft_model_id = 'mika5883/qwen3-4b_rugec'
# # peft_model_id = 'mika5883/ru_qwen7b_gec_Ag'
# # peft_model_id = 'mika5883/qwen3-14b_rugec'

# peft_model_id = 'mika5883/ru_qwen7b_gec_Ga' # you can try this one if you want to test on GERA since it's easy to get 

# # Load adapter config
# peft_config = PeftConfig.from_pretrained(peft_model_id)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
#     # bnb_4bit_compute_dtype=torch.bf16,
# )
# # Load base model
# base_model = AutoModelForCausalLM.from_pretrained(
#     peft_config.base_model_name_or_path,
#     trust_remote_code=True,
#     device_map="auto",
#     quantization_config=bnb_config,
#     low_cpu_mem_usage=True,
# )

# # Load tokenizer
# tokenizer = AutoTokenizer.from_pretrained(peft_config.base_model_name_or_path, trust_remote_code=True)

# # Merge LoRA adapter with base model
# model = PeftModel.from_pretrained(base_model, peft_model_id)
# model.config.use_cache = False

In [3]:
model_name = 'mika5883/qwen3-14b_rugec'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 512,
    # dtype = dtype,
    load_in_4bit = True,
    # padding_side='left'
)
FastLanguageModel.for_inference(model)
tokenizer.padding_side= 'left'

==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.325 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [03:07<00:00, 62.49s/it]
Unsloth 2025.5.7 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [ ]:
# test = 'data/rugec/RULEC-GEC.test.tsv'
test = 'data/rugec/RULEC-GEC.dev.tsv'
# test = 'data/rugec/GERA.test.tsv'
rulec_test = pd.read_csv(test, delimiter='\t')

In [ ]:
import warnings
warnings.filterwarnings("ignore", message="An output with one or more elements was resized") #got this message repeatedle when using qwen3.

In [ ]:
def generate_predictions(input_sentences, batch_size=64, max_new_tokens=512):
    all_predictions = []
    
    for i in tqdm(range(0, len(input_sentences), batch_size)):
        batch = input_sentences[i:i+batch_size]        
        
        prompts = [
            f"<|im_start|>system\nТы учитель русского языка, который проверяет эссе своих учеников.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: {sentence}\nИсправленное предложение:<|im_end|>\n<|im_start|>assistant\n"
            for sentence in batch
        ]
        
        inputs = tokenizer(
            prompts,
            return_tensors="pt",
            # padding=True,
            padding='max_length',
            truncation=True,
            max_length=512,
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=max_new_tokens,
                # num_beams=5,
                temperature=0.7, top_p=0.8, top_k=20, min_p=0,#  min_p=0,
                early_stopping=True,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )
        # print(outputs.shape)
        decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True, clean_up_tokenization_spaces=False)

        for full_output in decoded:
            if "Исправленное предложение:" in full_output:
                corrected = full_output.split("\n")[-1].strip()
            else:
                corrected = full_output.strip()
            all_predictions.append(corrected)

    return all_predictions



# Example usage
input_sentences = [
    "Он не был способен на найти решение.",
    "Это пример теста с ошибками.",
    "У меня был друг, который всегда говорил, что можно работать не напрягаясь и получать хорошие деньги, но мне казалось, что это просто шутка."
]

predictions = generate_predictions(input_sentences)
for sentence, prediction in zip(input_sentences, predictions):
    print(f"❌ Input: {sentence}")
    print(f"✅ Prediction: {prediction}\n")


100%|██████████| 1/1 [00:04<00:00,  4.13s/it]

❌ Input: Он не был способен на найти решение.
✅ Prediction: Он не был способен найти решение .

❌ Input: Это пример теста с ошибками.
✅ Prediction: Это пример теста с ошибками .

❌ Input: У меня был друг, который всегда говорил, что можно работать не напрягаясь и получать хорошие деньги, но мне казалось, что это просто шутка.
✅ Prediction: У меня был друг , который всегда говорил , что можно работать не напрягаясь и получать хорошие деньги , но мне казалось , что это просто шутка .



In [15]:
rulec_test['corrected_sent'] = generate_predictions(rulec_test['corrupt_sent'])

100%|██████████| 40/40 [17:44<00:00, 26.61s/it]


In [16]:
rulec_test

,corrupt_sent,correct_sent,corrected_sent
0,"Экскурсия прошла великолепно , Владимир Анатол...","Экскурсия прошла великолепно , Владимир Анатол...","Экскурсия прошла великолепно , Владимир Анатол..."
1,"Курсовая может быть о любой теме , которую обс...","Курсовая может быть по любой теме , которую об...","Курсовая может быть о любой теме , которую обс..."
2,"Я вижу , я слышу всё вокруг меня .","Я вижу , я слышу всё вокруг меня .","Я вижу , я слышу всё вокруг меня ."
3,Служить в Российской армии ( для мужчин ),Служить в Российской армии ( для мужчин ),Служить в Российской армии ( для мужчин )
4,Вряд ли все мир обеспокоился бы из-за бунта в ...,Вряд ли весь мир обеспокоился бы из-за бунта в...,Вряд ли весь мир обеспокоился бы из-за бунта в...
...,...,...,...
2495,Его маятник двигался взад и вперёд в одной пло...,Его маятник двигался взад и вперёд в одной пло...,Его маятник двигался взад и вперёд в одной пло...
2496,"Например , в настоящее время в Соединённых шта...","Например , в настоящее время в Соединённых Шта...","Например , в настоящее время в Соединённых шта..."
2497,"Иногда женщины делают вещи , которые очень раз...","Иногда женщины делают вещи , которые очень раз...","Иногда женщины делают вещи , которые очень раз..."
2498,30 . 11 . 2009,30 . 11 . 2009,30 . 11 . 2009


In [17]:
rulec_test = rulec_test.drop(columns=['correct_sent'])
rulec_test = rulec_test.rename(columns={'corrupt_sent' : 'corrupt', 'corrected_sent' : 'corrected'})

In [18]:
rulec_test

,corrupt,corrected
0,"Экскурсия прошла великолепно , Владимир Анатол...","Экскурсия прошла великолепно , Владимир Анатол..."
1,"Курсовая может быть о любой теме , которую обс...","Курсовая может быть о любой теме , которую обс..."
2,"Я вижу , я слышу всё вокруг меня .","Я вижу , я слышу всё вокруг меня ."
3,Служить в Российской армии ( для мужчин ),Служить в Российской армии ( для мужчин )
4,Вряд ли все мир обеспокоился бы из-за бунта в ...,Вряд ли весь мир обеспокоился бы из-за бунта в...
...,...,...
2495,Его маятник двигался взад и вперёд в одной пло...,Его маятник двигался взад и вперёд в одной пло...
2496,"Например , в настоящее время в Соединённых шта...","Например , в настоящее время в Соединённых шта..."
2497,"Иногда женщины делают вещи , которые очень раз...","Иногда женщины делают вещи , которые очень раз..."
2498,30 . 11 . 2009,30 . 11 . 2009


In [ ]:
rulec_test.to_csv('results/qwen2.5-14b_dev.tsv', sep='\t', index=False) #use your path